In [1]:
from monitor.monitor import Monitor
from monitor.lsa import LSA
from monitor.mdsa import MDSA

EXTRACT_NEW_TRACES_AND_PREDICTIONS = True
METRIC_TYPES = [ LSA, MDSA ]

In [2]:
monitor = Monitor()
print(monitor)

LUNAR LANDER MONITOR PROPERTIES:

	 Agent Model Path: 		./output/lunar_lander.keras
	 Agent Num Actions: 		4

	 Training Data Path: 		./output/latest_buffer_states.npy
	 Training Label Path: 		./output/latest_buffer_gt_actions.npy
	 Test Data Path: 		./input/randomly_generated.npy

	 Output Trace Path: 		./output/latest_traces_{}.npy
	 OUtput Prediction Path: 	./output/latest_preds.npy

	 Output Video Path: 		./output/lunar_lander.mp4
	 Output Video FPS: 			30


In [3]:
if EXTRACT_NEW_TRACES_AND_PREDICTIONS:
    monitor.extract_traces_and_preds()
else:
    print("Extracting new traces and predictions from scratch was not requested.")
# end if-else block

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1s 160us/step

Agent model Q-values saved to file: ./output/latest_preds.npy with shape: (100000, 4)
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1s 164us/step
Monitor traces for predicted action: 0 saved to file: ./output/latest_traces_0.npy with shape: (18985, 8)
Monitor traces for predicted action: 1 saved to file: ./output/latest_traces_1.npy with shape: (15271, 8)
Monitor traces for predicted action: 2 saved to file: ./output/latest_traces_2.npy with shape: (50517, 8)
Monitor traces for predicted action: 3 saved to file: ./output/latest_traces_3.npy with shape: (15227, 8)


In [4]:
print("TODO - generate some bogus data to use as test inputs")

TODO - generate some bogus data to use as test inputs


In [5]:
monitor.calculate_surprise_scores(METRIC_TYPES)

<class 'monitor.lsa.LSA'>
<class 'monitor.mdsa.MDSA'>


In [6]:
from monitor.monitor import DataGenerationType

data_gen_types = [ DataGenerationType.RANDOM, DataGenerationType.RANDOM_BOUNDED ]
monitor.generate_new_inputs(data_gen_types)

RANDOM: DataGenerationType.RANDOM
EVERYONE ELSE: DataGenerationType.RANDOM_BOUNDED
